## 무신사 스탠다드 리뷰 크롤링

**환경설정**

In [4]:
#필요하면 설치하기
#!pip install html_table_parser
#!pip install --upgrade beautifulsoup4

     -------------------------------------- 128.2/128.2 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.4.1
    Uninstalling beautifulsoup4-4.4.1:
      Successfully uninstalled beautifulsoup4-4.4.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
html-table-parser 0.1.0 requires beautifulsoup4==4.4.1, but you have beautifulsoup4 4.11.1 which is incompatible.


- BeautifulSoup 업데이트로 에러 발생
- 'AttributeError : module 'html5lib.treebuilders' has no attribute '_base''
- pip install --upgrade beautifulsoup4

In [2]:
import warnings
warnings.filterwarnings("ignore")
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re
import time
import pandas as pd

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from html_table_parser import parser_functions as parser

In [96]:
#함수정의:검색어 조건에 따른 url생성
#무신사 스탠다드->후기순->벨트 및 양말 등 잡화제외 1위 상품
def musinsa_searching(word):
  url = "https://www.musinsa.com/app/goods/1149328" 
  return url

In [4]:
from selenium.webdriver.common.keys import Keys
#가져오고싶은 상품 페이지 가져오기
#일단 후드-후기순-3번째페이지 링크 수기로 입력
url = 'https://www.musinsa.com/categories/item/001004?d_cat_cd=001004&brand=&list_kind=small&sort=emt_high&sub_sort=&page=3&display_cnt=90&group_sale=&exclusive_yn=&sale_goods=&timesale_yn=&ex_soldout=&kids=&color=&price1=&price2=&shoeSizeOption=&tags=&campaign_id=&includeKeywords=&measure='

from webdriver_manager.chrome import ChromeDriverManager
options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])

driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
driver.get(url)
time.sleep(5)

In [97]:
#각 리뷰에서 아래의 값들을 추출하는 함수 정의
#user, gender,height,weight,item, size,content, evaluation(size_eval,bright_eval,color_eval,thick_eval)
user_list = []
gender_list =[]
height_list = []
weight_list =[]
item_list = []
size_list = []
star_list= []
content_list = []
size_eval_list =[]
bright_eval_list =[]
color_eval_list =[]
thick_eval_list =[]

#함수정의
def get_content(driver):
    #함수안에 html, soup 넣어놔야 페이지 넘어가서 바르게 긁어옴, 밖에 빼놓으면 첫페이지만 여러번 긁어진다.
    html = driver.page_source
    soup = BeautifulSoup(html,'lxml')
    for i in range(10):
    
    #한 페이지당 10개의 리뷰씩 긁어와야하므로
    #profile(gender,height,weight)
    #남성, 177cm, 85kg

        profile_before = soup.find_all('p','review-profile__body_information')
        profile_after = profile_before[i].text.split(',')
        try:
            gender = profile_after[0]
            height = profile_after[1]
            weight = profile_after[2]
        except:
            gender = ''
            height = ''
            weight = ''
#user :LV 2 뉴비_95f88e16

           
#item: #/테이퍼드 히든 밴딩 크롭 슬랙스 [더스티 베이지]
#size :
#content

        try:
            user = soup.find_all('p','review-profile__name')[i].text
            item = soup.find_all('a','review-goods-information__name')[i].text
            # '/n' 없애고 추출하기
            size = soup.find_all('span', 'review-goods-information__option')[i].text.strip().replace('\n','')
            content = soup.find_all('div','review-contents__text')[i].text
        except:
            user = ''
            item = ''
            size = ''
            content = ''
            
        #star
#->별 5개일때:
#->별 4개일때:
        stars = driver.find_elements_by_xpath('//*[@id="reviewListFragment"]/div['+str(i+1)+']/div[3]/span/span/span')
        try:
            for j in stars:
                a =j.get_attribute('style')
                if a[7:9]=='20':
                    star = 1
                elif a[7:9]=='40':
                    star = 2
                elif a[7:9]=='60':
                    star = 3
                elif a[7:9]=='80':
                    star = 4
                else:
                    star = 5
        except:
            star = ''
      

    #evaluation
        evaluation = soup.find_all('div', 'review-evaluation')
        try:
            size_eval = evaluation[i].find_all('span')[0].text
            bright_eval = evaluation[i].find_all('span')[1].text
            color_eval = evaluation[i].find_all('span')[2].text
            thick_eval = evaluation[i].find_all('span')[3].text
        except:
            size_eval = ''
            bright_eval = ''
            color_eval = ''
            thick_eval = ''

#user,gender,height,weight,item, size,content,star,size_eval,bright_eval,color_eval,thick_eval
        
        user_list.append(user)
        gender_list.append(gender)
        height_list.append(height)
        weight_list.append(weight)
        item_list.append(item)
        size_list.append(size)
        content_list.append(content)
        star_list.append(star)
        size_eval_list.append(size_eval)
        bright_eval_list.append(bright_eval)
        color_eval_list.append(color_eval)
        thick_eval_list.append(thick_eval)
 

In [98]:
#버튼 누르기 함수정의
def move_next(driver):    
    for i in range(4):
        get_content(driver)
        #페이지 2,3,4,5 넘어가기
        driver.find_element_by_css_selector('#reviewListFragment > div.nslist_bottom > div.pagination.textRight > div > a:nth-child(' + 
                                            str(int(4) + int(i)) + ')').send_keys(Keys.ENTER)
        time.sleep(2)
    get_content(driver)
    
#그다음 화살표'>'버튼누르기: (6,7,8...)있는 페이지로 넘어가기   
def move_arrow(driver):
    driver.find_element_by_css_selector('#reviewListFragment > div.nslist_bottom > div.pagination.textRight > div > a.fa.fa-angle-right.paging-btn.btn.next').send_keys(Keys.ENTER)


**크롤링 시작**

In [287]:
from selenium.webdriver.common.keys import Keys
#가져오고싶은 상품 페이지 가져오기
#일단 후드-후기순-2번째페이지 url 수기로 입력
url = 'https://www.musinsa.com/categories/item/001004?d_cat_cd=001004&brand=&list_kind=small&sort=emt_high&sub_sort=&page=2&display_cnt=90&group_sale=&exclusive_yn=&sale_goods=&timesale_yn=&ex_soldout=&kids=&color=&price1=&price2=&shoeSizeOption=&tags=&campaign_id=&includeKeywords=&measure='

from webdriver_manager.chrome import ChromeDriverManager
options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
#알림창 닫는 옵션인데 안먹힘
#options.add_experimental_option('prefs', {'profile.default_content_setting_values.notifications' : 1})

driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
driver.get(url)
time.sleep(2)

In [288]:
#후기순 90개 자동으로 클릭하기
#뒤로가기 참고: https://m.blog.naver.com/mathesis_time/221970036063
# #searchList > li:nth-child(1) > div.li_inner > div.list_img > a > img
# #searchList > li:nth-child(2) > div.li_inner > div.list_img > a > img
# #searchList > li:nth-child(90) > div.li_inner > div.list_img > a > img
 #90개: 90
import math
import time

start = time.time()
math.factorial(100000)

#a값은 상품을 변경할때마다 변경해주어야함
a = 51
#상품개수에 따라 숫자 바꾸기
for i in range(70,90,1):
    driver.get(url)
    driver.find_element_by_css_selector('#searchList > li:nth-child(' +
                                            str(int(i)+int(1)) +
                                            ') > div.li_inner > div.list_img > a > img').click()
    time.sleep(2)
    try:
        driver.find_element_by_xpath('/html/body/div/div/div/button').click()
        driver.find_element_by_xpath('//*[@id="divpop_goods_niceghostclub_8451"]/form/button[2]').click()
    except:
        pass
    
    #사이즈표
    try:
        html = driver.page_source
        soup = BeautifulSoup(html,'html.parser')
        figure = soup.find('table',{'class':'table_th_grey'})
        p = parser.make2d(figure)
        figure_df = pd.DataFrame(data = p[1:],columns = p[0])
        figure_df.drop([0,1],inplace = True)
        print(figure_df)
    except:
        print("사이즈표 오류발생")
        
    #리뷰개수
    try:
        reviewNum = driver.find_element_by_css_selector('#estimate_style')
        reviewNum = reviewNum.text
        reviewNum = re.sub(r'[^0-9]','',reviewNum)
        reviewNum = int(reviewNum)
        print(reviewNum)
    except:
        print("리뷰개수 오류발생")
        

    user_list = []
    gender_list =[]
    height_list = []
    weight_list =[]
    item_list = []
    size_list = []
    star_list= []
    content_list = []
    size_eval_list =[]
    bright_eval_list =[]
    color_eval_list =[]
    thick_eval_list =[]
        
    #스타일후기리뷰 100개넘으면 진행,아니라면 진행 X
    if reviewNum >= 100:
        #크롤링
        #10:500개, 2:100개
        for i in range(2):
            try:
                move_next(driver)
                move_arrow(driver)
        #move_next(driver)
            except:
                time.sleep(2)

    else:
        driver.get(url)
        time.sleep(2)    
        
    globals()["df"+str(a)] = pd.DataFrame({'user':user_list,
                                           'gender':gender_list,
                                           'height':height_list,
                                           'weight':weight_list,
                                            'item':item_list,
                                            'size':size_list,
                                            'star':star_list,
                                            'content':content_list,
                                            'size_eval':size_eval_list,
                                            'bright_eval':bright_eval_list,
                                            'color_eval':color_eval_list,
                                            'thick_eval':thick_eval_list})          
    #사이즈표와 리뷰 merge
    time.sleep(2) 
    globals()["merge_df"+str(a)] = pd.merge(globals()["df"+str(a)],figure_df,how = 'left',left_on = 'size',right_on = 'cm')
    a = a + 1 
    
end = time.time()
print(f"{end - start:.5f} sec")

  cm    총장 어깨너비 가슴단면 소매길이
2  S  69.5   59   58   59
3  M    71   61   60   61
4  L  72.5   63   62   63
129
        cm  총장 어깨너비 가슴단면 소매길이
2        S  62   50   59   60
3        M  67   56   63   64
4        L  70   59   66   67
5  S (기모X)  62   50   59   60
6  M (기모X)  67   56   63   64
7  L (기모X)  70   59   66   67
2548
     cm    총장 어깨너비 가슴단면  소매길이
2  FREE  66.5   60   61  63.5
157
    cm  총장 어깨너비 가슴단면 소매길이
2    M  69   63   63   57
3    L  72   65   65   58
4   XL  75   67   67   59
5  2XL  78   69   69   60
404


TimeoutException: Message: timeout: Timed out receiving message from renderer: 600.000
  (Session info: chrome=106.0.5249.119)


**크롤링 리뷰 개수 확인**

In [290]:
print(merge_df71.shape)
print(merge_df72.shape)
print(merge_df73.shape)
print(merge_df74.shape)
print(merge_df67.shape)
print(merge_df68.shape)

NameError: name 'merge_df71' is not defined

In [249]:
#일단 1-31까지 100/99개 리뷰가 제대로 크롤링된 상품만 merge
page2 = pd.concat([merge_df3,merge_df4,merge_df5,merge_df6,merge_df7,
                  merge_df10, merge_df11,merge_df12,merge_df13,merge_df15,
                  merge_df16,merge_df17,merge_df18,merge_df19,
                  merge_df22, merge_df26,merge_df27,merge_df28,
                  merge_df29,merge_df30], axis=0)


#,merge_df33,merge_df34,merge_df35,merge_df36,merge_df37,merge_df38,merge_df39,
#                  merge_df40, merge_df41,merge_df42,merge_df43,merge_df44,merge_df45,merge_df46,merge_df47,merge_df48,merge_df49,
#                  merge_df50, merge_df51,merge_df52,merge_df53,merge_df54,merge_df55,merge_df56,merge_df57,merge_df58,merge_df59,
#                  merge_df60, merge_df61,merge_df62,merge_df63,merge_df64,merge_df65,merge_df66,merge_df67,merge_df68,merge_df69,
#                  merge_df70, merge_df71,merge_df72,merge_df73,merge_df74,merge_df75,merge_df76,merge_df77,merge_df78,merge_df79,
#                  merge_df80, merge_df81,merge_df82,merge_df83,merge_df84,merge_df85,merge_df86,merge_df87,merge_df88,merge_df89,
#                  merge_df90

In [264]:
page2 = pd.concat([page2, merge_df32, merge_df33, merge_df34, merge_df35,
                   merge_df39, merge_df40, merge_df42, merge_df43, merge_df46,
                   merge_df47, merge_df50], axis=0)


In [266]:
page2 = pd.concat([page2, merge_df32, merge_df33, merge_df34, merge_df35,
                   merge_df39, merge_df40, merge_df42, merge_df43, merge_df46,
                   merge_df47, merge_df50], axis=0).shape

(3098, 17)

In [286]:
merge_df70

,user,gender,height,weight,item,size,star,content,size_eval,bright_eval,color_eval,thick_eval,cm,총장,어깨너비,가슴단면,소매길이
0,LV 3 뉴비_8dc5896b60f8,여성,156cm,50kg,Scorpionida 자수 후드티 블랙,M,5,옷 짱짱 예쁘고 입을 맛 나요 잘 산 것 같아요 ㅎㅎ 잘 입겠슴둥,커요,보통이에요,보통이에요,얇아요,M,71,58,67,62
1,LV 5 삼각형과사각형,남성,172cm,60kg,Scorpionida 자수 후드티 블랙,M,5,남들과는 많이 안겹치는 후드티 찾다가 가성비도 좋아 구매했는데 만족도가 높습니다. ...,보통이에요,보통이에요,보통이에요,보통이에요,M,71,58,67,62
2,LV 3 뉴비_c948e3f095d9,남성,170cm,65kg,Scorpionida 자수 후드티 블랙,M,4,오버핏이라 핏이 이쁩니다 체형보완도 되고 다만 원단 퀄리티는 다소 떨어지는듯 ㅠㅠ,커요,보통이에요,보통이에요,얇아요,M,71,58,67,62
3,LV 3 서니와니지,여성,162cm,50kg,Scorpionida 자수 후드티 블랙,M,5,자수가 예뻐 망설임 없이 구매했는데 역시 과하지않고 예쁩니다.후드 모자도 모양이 ...,커요,보통이에요,선명해요,보통이에요,M,71,58,67,62
4,LV 3 뉴비_4a7ecdd8d640,여성,160cm,55kg,Scorpionida 자수 후드티 블랙,기모L,5,"오버핏을 좋아해서 L로 주문, 원하던 핏이에요. 기모 주문했지만 품절이라 연락받아서...",커요,어두워요,선명해요,두꺼워요,NaN,NaN,NaN,NaN,NaN
5,LV 3 뉴비_31f30b78,남성,173cm,70kg,Scorpionida 자수 후드티 블랙,M,5,자수가 과하지 않아서 좋음. 가격대비 디자인 너무 만족해요.,보통이에요,보통이에요,보통이에요,보통이에요,M,71,58,67,62
6,LV 3 뉴비_b65c64ff,여성,160cm,48kg,Scorpionida 자수 후드티 블랙,M,5,일단.. 자수가 너무 이쁘고 사이즈는 엉덩이 덮을만큼? 오버핏 사이즈예요. 스몰 있...,커요,보통이에요,보통이에요,보통이에요,M,71,58,67,62
7,LV 6 솔트뤼,남성,175cm,69kg,Scorpionida 자수 후드티 블랙,M,5,자수 너무 이뻐요 편하게 후드 입고 싶을때 잘 입을 것 같네요,보통이에요,보통이에요,보통이에요,보통이에요,M,71,58,67,62
8,LV 4 이리노일반인여자친구,여성,160cm,48kg,Scorpionida 자수 후드티 블랙,M,5,세탁 한 번 하면 냄새는 다 빠져요! 저 진짜 이 옷 너무너무 입고 싶어서 미칠 것...,보통이에요,보통이에요,선명해요,보통이에요,M,71,58,67,62
9,LV 4 신광일여친,여성,152cm,57kg,Scorpionida 자수 후드티 블랙,M,5,아주예뻐요앞모습을찍어놓고 뒷면을 칭찬하면 뭐지? 싶을텐데뒤의자수가진짜 캐캐예쁨^^하...,보통이에요,보통이에요,선명해요,얇아요,M,71,58,67,62


In [267]:
page2_re = pd.concat([merge_df1, merge_df2, merge_df8, merge_df9,
                   merge_df14, merge_df20, merge_df21, merge_df23, merge_df24,
                   merge_df25, merge_df31, merge_df36, merge_df37, merge_df38,
                  merge_df41, merge_df44, merge_df45, merge_df48, merge_df49], axis=0)

In [269]:
page2.to_csv('page2.csv', encoding='utf-8')

In [270]:
page2_re.to_csv('page2_re.csv', encoding='utf-8')

**창 닫기 시도**
- 창 개수 확인 : 현재 창이 여러개면 제일 처음 창만 두고 닫기 >> 그냥 하나의 창에 뜸 ^.^
- 에러(Alert)창 확인 : 경고창으로 뜨는 경우 닫아주는 함수 존재 >> 이것도 아님 ^.^
- 직접 버튼 지정해서 닫기 : driver.back 함수를 써야하는데, 버튼 닫을때 driver를 건들여서 넘어가지 않음 ^.^

In [59]:
#후기순 90개 자동으로 클릭하기
#뒤로가기 참고: https://m.blog.naver.com/mathesis_time/221970036063
# #searchList > li:nth-child(1) > div.li_inner > div.list_img > a > img
# #searchList > li:nth-child(2) > div.li_inner > div.list_img > a > img
# #searchList > li:nth-child(90) > div.li_inner > div.list_img > a > img
#90개: 90
a = 1
i = 0

#searchList > li:nth-child(1) > div.li_inner
##searchList > li:nth-child(1) > div.li_inner > div.list_img > a > img

for i in range(2):
    driver.get(url)
    driver.find_element_by_css_selector('#searchList > li:nth-child(' +
                                            str(int(1) + int(i)) +
                                            ') > div.li_inner > div.list_img > a > img').click()
    time.sleep(2)
        
    #사이즈표
    try:
        driver.find_element_by_xpath('/html/body/div/div/div/button').click()
        driver.find_element_by_xpath('//*[@id="divpop_goods_niceghostclub_8451"]/form/button[2]').click()
    except:
        pass
    try:
        html = driver.page_source
        soup = BeautifulSoup(html,'html.parser')
        figure = soup.find('table',{'class':'table_th_grey'})
        p = parser.make2d(figure)
        figure_df = pd.DataFrame(data = p[1:],columns = p[0])
        figure_df.drop([0,1],inplace = True)
        print(figure_df)
    except:
        print("사이즈표 오류발생")
    
    
    #리뷰개수
    try:
        driver.find_element_by_xpath('/html/body/div/div/div/button').click()
        driver.find_element_by_xpath('//*[@id="divpop_goods_niceghostclub_8451"]/form/button[2]').click()
    except:
        pass
    try:
        #기존코드 : reviewNum = driver.find_element_by_xpath('//*[@id="estimate_style"]')
        #변경코드 > content가 제대로 추출되지 않아서 변경함
        reviewNum = driver.find_element_by_css_selector('#estimate_style')
        reviewNum = reviewNum.text
        reviewNum = re.sub(r'[^0-9]','',reviewNum)
        reviewNum = int(reviewNum)
        print(reviewNum)
    except:
        print("리뷰개수 오류발생")
        
    user_list = []
    gender_list =[]
    height_list = []
    weight_list =[]
    item_list = []
    size_list = []
    star_list= []
    content_list = []
    size_eval_list =[]
    bright_eval_list =[]
    color_eval_list =[]
    thick_eval_list =[]
        
        #스타일후기리뷰 500개넘으면 진행,아니라면 진행 X
        #일단 100개 넘으면 진행하는 걸로 설정함
    if reviewNum >= 100:
        #크롤링
        #10:500개
        for i in range(1):
            try:
                move_next(driver)
                move_arrow(driver)
        #move_next(driver)
            except:
                time.sleep(2)
    else:
        driver.back()

   cm    총장  어깨너비  가슴단면 소매길이
2   S  64.5  53.5    64   64
3   M    67  55.5  66.5   65
4   L  69.5  57.5    69   66
5  XL    72  59.5  71.5   67
589


In [8]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#배송지연 메시지, 무료배송? 메시지 닫기
try:
        driver.find_element_by_xpath('/html/body/div/div/div/button').click()
        driver.find_element_by_xpath('//*[@id="divpop_goods_niceghostclub_8451"]/form/button[2]').click()
    except:
        pass